In [ ]:
import numpy as np
from source import *
from functools import reduce
from torch.utils.data import DataLoader

In [ ]:
def rot_flip(matrix1, matrix2):
    for k in range(4):
        rotated_matrix = np.rot90(matrix1, k=k)
        if np.array_equal(rotated_matrix, matrix2):
            return (k, -1)
        if np.array_equal(np.flip(rotated_matrix, axis=0), matrix2):
            return (k, 0)
        if np.array_equal(np.flip(rotated_matrix, axis=1), matrix2):
            return (k, 1)
        if np.array_equal(np.flip(rotated_matrix, axis=(0, 1)), matrix2):
            return (k, (0, 1))
    return None

In [ ]:
def transform_matrix(matrix, pivot, operation):
    height, width = matrix.shape
    px, py = pivot

    translated_matrix = np.zeros_like(matrix, dtype=int)
    for x in range(height):
        for y in range(width):
            new_x, new_y = x - px, y - py
            translated_matrix[new_x, new_y] = matrix[x, y]

    if operation == "rotate_90":
        result = np.rot90(translated_matrix, k=1)
    elif operation == "rotate_180":
        result = np.rot90(translated_matrix, k=2)
    elif operation == "rotate_270":
        result = np.rot90(translated_matrix, k=3)
    elif operation == "flip_horizontal":
        result = np.flip(translated_matrix, axis=1)
    elif operation == "flip_vertical":
        result = np.flip(translated_matrix, axis=0)
    else:
        raise ValueError(f"Unsupported operation: {operation}")

    final_matrix = np.zeros_like(matrix, dtype=int)
    for x in range(result.shape[0]):
        for y in range(result.shape[1]):
            original_x, original_y = x + px, y + py
            if 0 <= original_x < height and 0 <= original_y < width:
                final_matrix[original_x, original_y] = result[x, y]

    return final_matrix


def detect_transformation(matrix1, matrix2):
    operations = [
        "rotate_90", "rotate_180", "rotate_270",
        "flip_horizontal", "flip_vertical"
    ]

    for i in range(int(matrix1.shape[0])):
        for j in range(int(matrix1.shape[1])):
            for op1 in operations:
                transformed1 = transform_matrix(matrix1, (i, j), op1)
                if np.array_equal(transformed1, matrix2):
                    return (op1, None, (i, j))
                for op2 in operations:
                    transformed2 = transform_matrix(transformed1, (i, j), op2)
                    if np.array_equal(transformed2, matrix2):
                        return (op1, op2, (i, j))
    print('None')
    return None


In [ ]:
train_dataset = load_dataset(mode='evaluation')
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

problems = dict()

for _, data in enumerate(train_dataloader):
    id = data['id'][0]
    last_id = id
    train = data['train']
    test = data['test']
    problems[id] = dict()
    # relation_lst = []
    # seg_method = 'monochrome_contour'
    for i in range(len(train)):
        input_image = train[i]['input'].squeeze(0).numpy()
        output_image = train[i]['output'].squeeze(0).numpy()
        input_object = ARC_Object(image=input_image, mask=np.ones_like(input_image), parent=None)
        output_object = ARC_Object(image=output_image, mask=np.ones_like(output_image), parent=None)
        problems[id][f'ex_{i}'] = {'input': input_object, 'output': output_object}
        # input_extraction = extract_objects(input_object, method=seg_method)
        # output_extraction  = extract_objects(output_object, method=seg_method)
        # if len(input_extraction) > 10 or len(output_extraction) > 10:
        #     seg_method = 'color'
        #     input_extraction = extract_objects(input_object, method=seg_method)
        #     output_extraction  = extract_objects(output_object, method=seg_method)
    
        # problems[id][f'ex_{i}']['extracted'] = (input_extraction, output_extraction)
        # relation_lst.append(problems[id][f'ex_{i}']['extracted'])

    input_image = test[0]['input'].squeeze(0).numpy()
    output_image = test[0]['output'].squeeze(0).numpy()
    input_object = ARC_Object(image=input_image, mask=np.ones_like(input_image), parent=None)
    output_object = ARC_Object(image=output_image, mask=np.ones_like(output_image), parent=None)
    problems[id]['test'] = {'input': input_object, 'output': output_object}
    # input_extraction = extract_objects(input_object, method=seg_method)
    # output_extraction  = extract_objects(output_object, method=seg_method)
    # problems[id]['test']['extracted'] = (input_extraction, output_extraction)

In [ ]:
count = 0
for pid, prob in problems.items():
    op = rot_flip(prob['ex_0']['input'].grid, prob['ex_0']['output'].grid)
    if op is not None:
        new_obj = rotate(prob['test']['input'], op[0])
        if op[1] != -1:
            new_obj = flip(new_obj, axis=op[1])
        if np.array_equal(new_obj.grid, prob['test']['output'].grid):
            count += 1
            print(pid)
        else:
            print('Doubt reality')
            print(pid)
            print(op)
            new_obj.plot_grid()
            prob['test']['output'].plot_grid()
print(count)

In [ ]:
train_objs, test_obj = quick_load('90347967', 'evaluation')
res = detect_transformation(test_obj['input'].grid, test_obj['output'].grid)
print(res)
for o in train_objs:
    o['input'].plot_grid()
    o['output'].plot_grid()
# test_obj['input'].plot_grid()
# test_obj['output'].plot_grid()

In [ ]:
def same_obj(obj1: ARC_Object, obj2: ARC_Object):
    if obj1 is None or obj2 is None:
        return False
    return np.array_equal(obj1.grid, obj2.grid)

class ListProperties:
    def __init__(self, objs: List[ARC_Object]):
        self.objs = sorted(objs, key=lambda o: o.top_left)
        self.num_objs = len(self.objs)
        self.most_common = most_common(self.objs)
        self.majority = majority(self.objs)
        self.and_all = reduce(and_obj, self.objs)
        self.or_all = reduce(or_obj, self.objs)
        self.xor_all = reduce(xor_obj, self.objs)

class CompareObjects:
    def __init__(self, obj1: ARC_Object, obj2: ARC_Object):
        self.obj1 = obj1
        self.obj2 = obj2
        self.same_grid = same_obj(obj1, obj2)
        self.same_pos = obj1.top_left == obj2.top_left
        self.same_color = dominant_color(obj1) == dominant_color(obj2)
        self.same_size = (obj1.height == obj2.height) and (obj1.width == obj2.width)
        self.rot_flip = rot_flip(obj1.grid, obj2.grid)

class CompareLists:
    def __init__(self, lst1: ListProperties, lst2: ListProperties):
        self.lst1 = lst1
        self.lst2 = lst2
        self.compare_objs = [CompareObjects(o1, o2) for o1, o2 in zip(lst1.objs, lst2.objs)]
        self.same_len = lst1.num_objs == lst2.num_objs
        self.same_most_common = same_obj(lst1.most_common, lst2.most_common)
        self.same_majority = same_obj(lst1.majority, lst2.majority)
        self.same_and = same_obj(lst1.and_all, lst2.and_all)
        self.same_or = same_obj(lst1.or_all, lst2.or_all)
        self.same_xor = same_obj(lst1.xor_all, lst2.xor_all)

In [ ]:
# train_objs, test_obj = quick_load('1caeab9d', 'training')
# in_children = extract_objects(train_objs[0]['input'], method='color')
# out_children = extract_objects(train_objs[0]['output'], method='color')
# train_objs[0]['input'].plot_grid()
# print(train_objs[0]['input'].grid)
# matrix = train_objs[0]['input'].grid
# print(matrix.T)
# new_obj = ARC_Object(matrix.T, np.ones_like(matrix.T))
# new_obj.plot_grid()
# train_objs[0]['output'].plot_grid()

# in1_p = ListProperties(in_children)
# out1_p = ListProperties(out_children)

# analyze1 = CompareLists(in1_p, out1_p)

# for r in analyze1.compare_objs:
#     r.obj1.plot_grid()
#     r.obj2.plot_grid()
#     print(r.obj1.top_left, r.obj2.top_left)
#     print(r.same_grid, r.same_pos, r.same_color, r.same_size)

# print(analyze1.same_len, analyze1.same_most_common, analyze1.same_majority, analyze1.same_and, analyze1.same_or, analyze1.same_xor)